In [1]:
import pandas as pd
import requests as rq
import numpy as np

In [2]:
#nyc complains data: 
complains_url="https://data.cityofnewyork.us/api/views/erm2-nwe9/rows.csv"
pop_byzip_url = "https://data.cityofnewyork.us/resource/rreq-n6zk.csv"
pop_byborough_url = "https://data.cityofnewyork.us/api/views/h2bk-zmw6/rows.csv?accessType=DOWNLOAD"

#### for some reason pandas ignores nrows() when reading from URL so we use bash magic to expolore schema

In [3]:
%%bash -s "$complains_url" "$pop_byzip_url" "$pop_byborough_url"
echo "++++++++++++++++++++++++++++++ COMPLAINS ++++++++++++++++++++++++++++++++"
wget -qO- $1 | head -2

echo "++++++++++++++++++++++++ POPULATION BY ZIP CODES ++++++++++++++++++++++++"
wget -qO- $2 | head -2

echo "+++++++++++++++++++++++++ POPULATION BY BOROUGH +++++++++++++++++++++++++"
wget -qO- $3 | head -2

++++++++++++++++++++++++++++++ COMPLAINS ++++++++++++++++++++++++++++++++
Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Description,Resolution Action Updated Date,Community Board,BBL,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Park Facility Name,Park Borough,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
16801224,06/04/2010 09:24:00 AM,06/10/2010 12:00:00 PM,DSNY,P - Manhattan and Bronx,Dirty Conditions,E12 Illegal Dumping Surveillance,Sidewalk,10003,27 2 AVENUE,2 AVENUE,EAST 1 STREET,EAST 2 STREET,,,ADDRESS,NEW YORK,,N/A,Closed,,"The Department of Sanitation has investigated the complaint and addressed 

In [95]:
%%bash -s "$complains_url" "$pop_byzip_url" "$pop_byborough_url"
[ -f data/complains.csv ] || wget -O data/complains.csv $2 > /dev/null
[ -f data/popul_by_zip.csv ] || wget -O data/popul_by_zip.csv $2 > /dev/null
[ -f data/popul_by_borough.csv ] || wget -O data/popul_by_borough.csv $2 > /dev/null

### Data loading, transforming & exploration

In [4]:
complains_f = "data/complains.csv"
pop_byzip_f = "data/popul_by_zip.csv"
pop_byborough_f = "data/popul_by_borough.csv"

usecols = ['Unique Key','Created Date','Borough','Incident Zip','City','Complaint Type']
dtypes = {'Unique Key':np.uint32,'Incident Zip':'category','Borough':'category','City':'category','Complaint Type':'category'}
a_values=[0, "0", "N/A", 0, "N/A", "N/A"]

chunksize = 200000
year=2017

In [5]:
df_iter = pd.read_csv(complains_f, skipinitialspace=True, index_col='Unique Key',
                         usecols=usecols,
                         dtype=dtypes,
                         na_values=na_values,
                         low_memory=False, iterator=True, chunksize=chunksize)

In [6]:
# Filter by year of interest: 2017 and remove 'Borough' == "Unspecified"
# We read in chunks to keep memory consumption controlled iterator=True
# Issue: iterator=True doesn't return iterable immediately for URL resources and reads whole resource into memory
# Issue: ValueError: could not convert string to float: '11219-0283'
# Issue: ValueError: Integer column has NA values in column

ch_bad = []
chunks = []
for chunk in df_iter:
    c = None
    try:
        c = chunk[(pd.to_datetime(chunk['Created Date']).dt.year == year) & (chunk['Borough'] != "Unspecified")]
# lambda: c = chunk.pipe(lambda x: x[(pd.to_datetime(x['Created Date']).dt.year == year) & (chunk['Borough'] != "Unspecified")])
        if (len(c) > 0):
            chunks.append(c)
            print("Added %d rows" %(len(c)))
    except:
        print("Failure in processing a chunk:")
        print(c)

Added 1 rows
Added 75223 rows
Added 97416 rows
Added 98219 rows
Added 98242 rows
Added 98521 rows
Added 98278 rows
Added 98363 rows
Added 98387 rows
Added 98410 rows
Added 98369 rows
Added 98425 rows
Added 98871 rows
Added 98853 rows
Added 98717 rows
Added 98627 rows
Added 98882 rows
Added 98827 rows
Added 98118 rows
Added 97570 rows
Added 77220 rows
Added 4 rows
Added 1 rows
Added 1849 rows
Added 2827 rows
Added 1651 rows
Added 1479 rows
Added 1804 rows
Added 1883 rows
Added 1620 rows
Added 2212 rows
Added 2397 rows
Added 2528 rows
Added 1708 rows
Added 810 rows
Added 1278 rows
Added 1677 rows
Added 2046 rows
Added 2078 rows
Added 2250 rows
Added 2631 rows
Added 2339 rows
Added 2427 rows
Added 2445 rows
Added 2007 rows
Added 1919 rows
Added 1910 rows
Added 2018 rows
Added 2013 rows
Added 1620 rows
Added 2188 rows
Added 1520 rows
Added 1238 rows
Added 1426 rows
Added 1519 rows
Added 1094 rows
Added 1481 rows
Added 1472 rows
Added 1219 rows
Added 402 rows
Added 500 rows
Added 66 rows
Ad

In [13]:
df = pd.concat(chunks, axis=0, ignore_index=False)

In [17]:
df.head(n=5)

,Created Date,Complaint Type,Incident Zip,City,Borough
Unique Key,,,,,
34912416,01/03/2017 09:12:00 AM,Vacant Lot,11211,BROOKLYN,BROOKLYN
35108046,03/30/2017 07:03:00 AM,Vacant Lot,11211,BROOKLYN,BROOKLYN
35117399,02/21/2017 10:49:00 AM,Vacant Lot,10305,STATEN ISLAND,STATEN ISLAND
35123353,02/06/2017 11:47:00 AM,Vacant Lot,11208,BROOKLYN,BROOKLYN
35136920,01/01/2017 12:31:58 AM,Noise - Residential,11373,ELMHURST,QUEENS


In [16]:
df.dtypes

Created Date        object
Complaint Type      object
Incident Zip        object
City                object
Borough           category
dtype: object

In [34]:
#df['Incident Zip'].replace(to_replace="\$([0-9,\.]+).*", value=r"\1", regex=True, inplace=True)
df['Incident Zip'] = df['Incident Zip'].apply(lambda x: "0" if str(x) in ["nan", "UNKNOWN", "."] else str(x).replace(r'\s*-.*', '')).astype(np.uint16)


In [32]:
df.to_csv("data/complains_2017.csv", index=True)

In [35]:
df.dtypes

Created Date        object
Complaint Type      object
Incident Zip        uint16
City                object
Borough           category
dtype: object

In [36]:
df.describe(include = 'all')

,Created Date,Complaint Type,Incident Zip,City,Borough
count,2403938,2403938,2.403938e+06,2338694,2403938
unique,1798689,205,NaN,124,5
top,02/15/2017 12:00:00 AM,Noise - Residential,NaN,BROOKLYN,BROOKLYN
freq,273,229458,NaN,746206,760965
mean,NaN,NaN,1.053876e+04,NaN,NaN
std,NaN,NaN,1.838529e+03,NaN,NaN
min,NaN,NaN,0.000000e+00,NaN,NaN
25%,NaN,NaN,1.030900e+04,NaN,NaN
50%,NaN,NaN,1.120400e+04,NaN,NaN
75%,NaN,NaN,1.123400e+04,NaN,NaN


In [37]:
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
UInt64Index: 2403938 entries, 34912416 to 34708823
Data columns (total 5 columns):
Created Date      2403938 non-null object
Complaint Type    2403938 non-null object
Incident Zip      2403938 non-null uint16
City              2338694 non-null object
Borough           2403938 non-null category
dtypes: category(1), object(3), uint16(1)
memory usage: 80.2+ MB


In [84]:
df_borough = df[['Borough']].drop_duplicates()
df_borough

,Borough
Unique Key,
34912416,BROOKLYN
35117399,STATEN ISLAND
35136920,QUEENS
35136936,MANHATTAN
35137010,BRONX


In [83]:
df.groupby(['Borough'])['Borough'].count().reset_index(name='cnt').sort_values(by='cnt', ascending=False)
# Why Unspecified showed up?

,Borough,cnt
1,BROOKLYN,760965
3,QUEENS,586910
2,MANHATTAN,480298
0,BRONX,449048
4,STATEN ISLAND,126717
5,Unspecified,0


In [41]:
df_compl_type = df[['Complaint Type']].drop_duplicates()
df_compl_type

,Complaint Type
Unique Key,
34912416,Vacant Lot
35136920,Noise - Residential
35136927,Derelict Vehicle
35136935,Noise - Commercial
35136950,Blocked Driveway
35137486,Noise - Street/Sidewalk
35137488,Drug Activity
35137499,Smoking
35137517,Noise - Helicopter


In [45]:
df_city = df[['City']].drop_duplicates()
print(df_city)

                           City
Unique Key                     
34912416               BROOKLYN
35117399          STATEN ISLAND
35136920               ELMHURST
35136927         MIDDLE VILLAGE
35136936               NEW YORK
35136964    SOUTH RICHMOND HILL
35136975                JAMAICA
35136977               FLUSHING
35136999          FRESH MEADOWS
35137010                  BRONX
35137369              REGO PARK
35137391          ROCKAWAY PARK
35137454          RICHMOND HILL
35137515                ASTORIA
35137545                 CORONA
35137591              RIDGEWOOD
35137624                MASPETH
35137863       SOUTH OZONE PARK
35138311              SUNNYSIDE
35138401           FAR ROCKAWAY
35138406                ARVERNE
35138518          EAST ELMHURST
35138546              WOODHAVEN
35138810                BAYSIDE
35144828          College Point
35139244               WOODSIDE
35139267             OZONE PARK
35139580            KEW GARDENS
35139665               ROSEDALE
35140457

In [42]:
# Borough == Unspecified is less then 0.5% of all records so we are going to drop those records from the DataFrame
# This also can be done at a loading time (see above)
df = df[df['Borough'].map(lambda x: str(x) != "Unspecified")]
df.count()

Created Date      2403938
Complaint Type    2403938
Incident Zip      2403938
City              2338694
Borough           2403938
dtype: int64

In [46]:
df.groupby(['Borough'])['Borough'].count().reset_index(name='cnt').sort_values(by='cnt', ascending=False)

,Borough,cnt
1,BROOKLYN,760965
3,QUEENS,586910
2,MANHATTAN,480298
0,BRONX,449048
4,STATEN ISLAND,126717
5,Unspecified,0


In [48]:
# There are few complains where City or/and Zip is Nan:
city_null = df['City'].isnull()
zip_null = df['Incident Zip'].isnull()

# Select these cases:
df[zip_null | city_null]

,Created Date,Complaint Type,Incident Zip,City,Borough
Unique Key,,,,,
35143285,01/01/2017 12:34:00 AM,Street Light Condition,0,NaN,MANHATTAN
35143942,01/01/2017 08:21:00 PM,Street Light Condition,0,NaN,QUEENS
35145371,01/01/2017 02:10:00 PM,Traffic Signal Condition,0,NaN,MANHATTAN
35144241,01/01/2017 02:35:00 PM,Traffic Signal Condition,0,NaN,MANHATTAN
35144244,01/01/2017 07:10:00 PM,Traffic Signal Condition,0,NaN,MANHATTAN
35144533,01/01/2017 09:03:00 PM,Street Light Condition,0,NaN,BRONX
35144534,01/01/2017 11:47:00 AM,Street Light Condition,0,NaN,MANHATTAN
35144582,01/01/2017 05:51:00 PM,Street Light Condition,0,NaN,MANHATTAN
35144602,01/01/2017 08:05:56 PM,Highway Condition,0,NaN,QUEENS


In [50]:
df = df[df['Incident Zip'].notnull() | df['City'].notnull()]
df.count()

Created Date      2403938
Complaint Type    2403938
Incident Zip      2403938
City              2338694
Borough           2403938
dtype: int64

### Assessment part

#### 1. Consider only the 10 most common overall complaint types. For each borough, how many of each of those 10 types were there in 2017?

In [51]:
# First top 10 complains types:
top10_complains = df.groupby(['Complaint Type'])['Complaint Type'].count().reset_index(name='cnt').sort_values(by='cnt', ascending=False).head(n=10)
top10_complains

,Complaint Type,cnt
129,Noise - Residential,229458
87,HEAT/HOT WATER,213521
104,Illegal Parking,144828
27,Blocked Driveway,135059
171,Street Condition,93222
172,Street Light Condition,82807
189,UNSANITARY CONDITION,79282
130,Noise - Street/Sidewalk,72868
202,Water System,65100
124,Noise,60168


In [52]:
all_merged_with_top10 = pd.merge(top10_complains, df, on='Complaint Type')

In [53]:
# For each borough, how many of each of those 10 types were there in 2017?
top10_borough_complains = all_merged_with_top10.groupby(['Borough','Complaint Type'])['Complaint Type'].count().reset_index(name='cnt').sort_values(by=['Borough','cnt'], ascending=[True, False])
top10_borough_complains

,Borough,Complaint Type,cnt
1,BRONX,HEAT/HOT WATER,68718
4,BRONX,Noise - Residential,57663
0,BRONX,Blocked Driveway,24574
8,BRONX,UNSANITARY CONDITION,24561
7,BRONX,Street Light Condition,18410
2,BRONX,Illegal Parking,16122
5,BRONX,Noise - Street/Sidewalk,14025
6,BRONX,Street Condition,11761
9,BRONX,Water System,10221
3,BRONX,Noise,3134


In [54]:
# top 10 complains by each borough (this contains exaclt 10 rows per borough):
top10_complains_by_borough = df.groupby(['Borough','Complaint Type'])['Complaint Type'].count().groupby(level='Borough').nlargest(10).reset_index(level=0, drop=True)
top10_complains_by_borough

Borough        Complaint Type                     
BRONX          HEAT/HOT WATER                         68718
               Noise - Residential                    57663
               Blocked Driveway                       24574
               UNSANITARY CONDITION                   24561
               PAINT/PLASTER                          19496
               Street Light Condition                 18410
               PLUMBING                               16490
               Illegal Parking                        16122
               Noise - Street/Sidewalk                14025
               DOOR/WINDOW                            11850
BROOKLYN       Noise - Residential                    67629
               HEAT/HOT WATER                         66984
               Illegal Parking                        55380
               Blocked Driveway                       49302
               UNSANITARY CONDITION                   26659
               Street Condition                  

#### 2. Consider only the 10 most common overall complaint types. For the 10 most populous zip codes, how many of each of those 10 types were there in 2017?


In [56]:
#1. Find top 10 mst populous zip codes
top10_zip_pop = pd.read_csv(pop_byzip_f, skipinitialspace=True, usecols=['jurisdiction_name','count_participants']).sort_values(by=['count_participants'], ascending=[False]).head(10)
top10_zip_pop

,count_participants,jurisdiction_name
232,272,12789
218,252,12734
130,248,11230
228,242,12779
120,214,11219
229,201,12783
222,134,12754
210,124,12428
119,111,11218
124,109,11223


In [58]:
t_df = pd.merge(top10_zip_pop, df, left_on="jurisdiction_name", right_on='Incident Zip')

In [59]:
top10_complains_by_zip = t_df.groupby(['Incident Zip','Complaint Type'])['Complaint Type'].count().groupby(level='Incident Zip').nlargest().reset_index(level=0, drop=True)
top10_complains_by_zip

Incident Zip  Complaint Type                     
11218         Illegal Parking                        2483
              Blocked Driveway                       1751
              Noise - Residential                    1621
              HEAT/HOT WATER                         1425
              UNSANITARY CONDITION                    649
11219         Illegal Parking                        1911
              Blocked Driveway                       1477
              HEAT/HOT WATER                          961
              Request Large Bulky Item Collection     866
              Street Condition                        811
11223         Illegal Parking                        3073
              Blocked Driveway                       1758
              Noise - Residential                    1015
              HEAT/HOT WATER                          999
              Street Condition                        861
11230         Illegal Parking                        2435
              HEAT/HOT

#### 3. Considering all complaint types. Which boroughs are the biggest "complainers" relative to the size of the population in 2017 Meaning, calculate a complaint-index that adjusts for population of the borough

In [78]:
borough_popul = pd.read_csv(pop_byborough_f, dtype={'Population':np.uint32,'Borough':'category'})
borough_popul['Borough'] = borough_popul['Borough'].apply(lambda x: x.upper())
borough_popul

,Borough,Population
0,BROOKLYN,4970026
1,MANHATTAN,3123068
2,BRONX,2717758
3,QUEENS,4460101
4,STATEN ISLAND,912458


In [79]:
borough_popul.dtypes

Borough       category
Population      uint32
dtype: object

In [80]:
borough_compl = df.groupby(['Borough'])['Borough'].count().reset_index(name='complains')
borough_compl

,Borough,complains
0,BRONX,449048
1,BROOKLYN,760965
2,MANHATTAN,480298
3,QUEENS,586910
4,STATEN ISLAND,126717
5,Unspecified,0


In [81]:
borough_popul.dtypes

Borough       category
Population      uint32
dtype: object

In [82]:
borough_compl_per_popul_unit = pd.merge(borough_compl, borough_popul, on="Borough")
borough_compl_per_popul_unit['compl_per_1m_popul'] = 1000000*borough_compl_per_popul_unit['complains']/borough_compl_per_popul_unit['Population']
borough_compl_per_popul_unit = borough_compl_per_popul_unit.sort_values(by=['compl_per_1m_popul'], ascending=[False])
borough_compl_per_popul_unit

,Borough,complains,Population,compl_per_1m_popul
0,BRONX,449048,2717758,165227.367558
2,MANHATTAN,480298,3123068,153790.439401
1,BROOKLYN,760965,4970026,153110.869038
4,STATEN ISLAND,126717,912458,138874.337230
3,QUEENS,586910,4460101,131591.190424


In [ ]:
# tips:
# pd.set_option('display.max_rows', 1000)
# pd.set_option('display.max_columns', 100)